## Goal of Project: 
-- predict whether a restaurant will die in a year based on information of permenantly closed restaurants in chicago

## Proposed methods:
-- collect info of both permenantly closed restaurants and those remain open in the same period of time and in certain neighbothoods of chicago as the data 
## Proposed model:
-- Random Forests Classifier

## Risk:
-- Can't possibly obtain the full set of restaurants for both closed ones and open ones in chicago based on Yelp

## Assumptions:
-- Most signs of a restaurant not going well can be captured from its Yelp data

## Initial Goals:
-- beat the baseline

In [352]:
import pandas as pd
import numpy as np

In [353]:
from os import listdir
from os.path import isfile, join

In [354]:
def Getdf(path='./Gayot/'):
    files = [f for f in listdir(path)]
    Reviewfiles = [f for f in files if '_review.csv' in f]
    Infofiles = [f[:-11]+'.csv' for f in Reviewfiles if f[:-11]+'.csv' in files]
    df = pd.read_csv(path + Infofiles[0])
    df['id'] = Infofiles[0][:-4]
    try:
        review = pd.read_csv(path+Infofiles[0][:-4]+'_review.csv')
        df['first_review'] = [review['date'].iloc[-1]]
        df['last_review'] = [review['date'].iloc[0]]
    except:
        df['first_review'] = ['MISSING']
        df['last_review'] = ['MISSING']        
        #print('Review file error for '+path+Infofiles[0])    
    for f in Infofiles[1:]:
        newdf = pd.read_csv(path+f)
        newdf['id'] = f[:-4]
        try:
            review = pd.read_csv(path+f[:-4]+'_review.csv')
            newdf['first_review'] = [review['date'].iloc[-1]]
            newdf['last_review'] = [review['date'].iloc[0]]        
        except:
            newdf['first_review'] = ['MISSING']
            newdf['last_review'] = ['MISSING']        
        df = df.append(newdf, ignore_index=True)            
    return df

In [355]:
Gayot = Getdf(path='./Gayot/')
Gayot.head()

,address,category,claimed_status,first_review,health_rating,id,info,last_review,latitude,longitude,...,neighborhood,permanently_closed,phone,price_range,ratings,ratings_histogram,reviews,url,website,working_hours
0,"553 West Diversey Pkwy Chicago, IL 60614 b/t H...","Breakfast & Brunch,American (Traditional)",Claimed,2011-10-08,NaN,0_2-sparrows,"[{'Takes Reservations': 'No'}, {'Delivery': 'N...",2015-02-02,41.932572,-87.643253,...,Lincoln Park,1,(773) 234-2320,$11-30,3.0,"[{5: 63}, {4: 94}, {3: 67}, {2: 78}, {1: 34}]",336.0,https://www.yelp.com/biz/2-sparrows-chicago,http://2sparrowschicago.com,[]
1,"140 E Walton St Chicago, IL 60611 b/t Michigan...","Lounges,American (Traditional)",Claimed,2006-06-23,NaN,100_coq-d-or-restaurant-and-lounge,"[{'Takes Reservations': 'No'}, {'Delivery': 'N...",2017-07-31,41.900147,-87.623480,...,"Near North Side, Streeterville",0,(312) 787-2200,$11-30,3.5,"[{5: 39}, {4: 41}, {3: 24}, {2: 16}, {1: 11}]",131.0,NaN,NaN,"[{'Mon': '11:00 am - 1:00 am'}, {'Tue': '11:00..."
2,"2111 W Armitage Ave Chicago, IL 60647 b/t Leav...","Cuban,Latin American",Claimed,2005-08-19,NaN,101_cafe-laguardia,"[{'Takes Reservations': 'Yes'}, {'Delivery': '...",2014-06-08,41.917521,-87.680450,...,Bucktown,1,(773) 862-5996,$11-30,3.5,"[{5: 95}, {4: 187}, {3: 81}, {2: 43}, {1: 27}]",433.0,https://www.yelp.com/biz/cafe-laguardia-chicago,http://www.cafelaguardia.com,[]
3,"871 N Rush St Chicago, IL 60686 Near North Side",Italian,NaN,2005-11-02,NaN,103_cafe-luciano,[],2015-10-15,41.898765,-87.626551,...,Near North Side,1,(312) 266-1414,NaN,3.5,"[{5: 1}, {4: 3}, {3: 0}, {2: 0}, {1: 1}]",5.0,https://www.yelp.com/biz/cafe-luciano-chicago,NaN,[]
4,"1846 N Milwaukee Ave Chicago, IL 60647 b/t Blo...",French,Claimed,2005-11-08,NaN,104_cafe-matou,"[{'Takes Reservations': 'Yes'}, {'Delivery': '...",2011-04-12,41.914953,-87.684838,...,Bucktown,1,(773) 384-8911,$31-60,4.0,"[{5: 30}, {4: 44}, {3: 15}, {2: 8}, {1: 2}]",99.0,https://www.yelp.com/biz/cafe-matou-chicago,http://www.cafematou.com,[]


In [356]:
Tribune = Getdf(path='./Tribune/')
Tribune.head()

,address,category,claimed_status,first_review,health_rating,id,info,last_review,latitude,longitude,...,neighborhood,permanently_closed,phone,price_range,ratings,ratings_histogram,reviews,url,website,working_hours
0,"736 N Clark St Chicago, IL 60654 Near North Si...",Burgers,Claimed,2011-08-29,NaN,0_25-degrees,"[{'Takes Reservations': 'No'}, {'Delivery': 'N...",2017-08-21,41.895943,-87.631593,...,"Near North Side, River North",0,(312) 943-9700,$11-30,4.0,"[{5: 275}, {4: 282}, {3: 89}, {2: 39}, {1: 32}]",717.0,https://www.yelp.com/biz/25-degrees-chicago,http://25degreesrestaurant.com/chicago/,"[{'Mon': '11:00 am - 11:00 pm'}, {'Tue': '11:0..."
1,"1252 N Wells St Chicago, IL 60610 b/t Scott St...",Mexican,Claimed,2005-07-31,NaN,100_salpicon,"[{'Takes Reservations': 'Yes'}, {'Delivery': '...",2016-12-16,41.905413,-87.634694,...,"Near North Side, Old Town",1,(312) 988-7811,$31-60,3.5,"[{5: 80}, {4: 75}, {3: 46}, {2: 35}, {1: 54}]",290.0,https://www.yelp.com/biz/salpicon-chicago,http://www.salpicon.com/,[]
2,"2575 N Lincoln Ave Chicago, IL 60614 b/t Sheff...",Diners,NaN,2005-07-13,NaN,101_salt-and-pepper-diner,"[{'Takes Reservations': 'No'}, {'Delivery': 'N...",2013-11-10,41.928779,-87.653220,...,"Lincoln Park, DePaul",1,(773) 525-8788,Under $10,4.0,"[{5: 36}, {4: 43}, {3: 14}, {2: 4}, {1: 4}]",101.0,https://www.yelp.com/biz/salt-and-pepper-diner...,NaN,[]
3,"2375 W Armitage Ave Chicago, IL 60647 b/t West...",Hot Dogs,NaN,2006-12-27,NaN,102_sams-red-hots,"[{'Takes Reservations': 'No'}, {'Delivery': 'N...",2016-10-11,41.917496,-87.686614,...,Bucktown,1,NaN,Under $10,3.5,"[{5: 20}, {4: 9}, {3: 13}, {2: 7}, {1: 10}]",59.0,NaN,NaN,[]
4,"444 W Fullerton Pkwy Chicago, IL 60614 b/t Lak...","Breakfast & Brunch,Coffee & Tea",Claimed,2009-07-17,NaN,103_sarks-in-the-park,"[{'Takes Reservations': 'Yes'}, {'Delivery': '...",2016-09-10,41.925894,-87.641075,...,Lincoln Park,1,(773) 404-9000,$11-30,2.5,"[{5: 30}, {4: 32}, {3: 45}, {2: 44}, {1: 99}]",250.0,https://www.yelp.com/biz/sarks-in-the-park-chi...,http://www.sarksinthepark.com/,[]


In [357]:
Gayot.shape

(411, 21)

In [358]:
Tribune.shape

(150, 21)

# DataFrame of most closed restaurants from Gayot.com and Tribune

In [359]:
Res = Gayot.append(Tribune, ignore_index=True)

In [360]:
Res

,address,category,claimed_status,first_review,health_rating,id,info,last_review,latitude,longitude,...,neighborhood,permanently_closed,phone,price_range,ratings,ratings_histogram,reviews,url,website,working_hours
0,"553 West Diversey Pkwy Chicago, IL 60614 b/t H...","Breakfast & Brunch,American (Traditional)",Claimed,2011-10-08,NaN,0_2-sparrows,"[{'Takes Reservations': 'No'}, {'Delivery': 'N...",2015-02-02,41.932572,-87.643253,...,Lincoln Park,1,(773) 234-2320,$11-30,3.0,"[{5: 63}, {4: 94}, {3: 67}, {2: 78}, {1: 34}]",336.0,https://www.yelp.com/biz/2-sparrows-chicago,http://2sparrowschicago.com,[]
1,"140 E Walton St Chicago, IL 60611 b/t Michigan...","Lounges,American (Traditional)",Claimed,2006-06-23,NaN,100_coq-d-or-restaurant-and-lounge,"[{'Takes Reservations': 'No'}, {'Delivery': 'N...",2017-07-31,41.900147,-87.623480,...,"Near North Side, Streeterville",0,(312) 787-2200,$11-30,3.5,"[{5: 39}, {4: 41}, {3: 24}, {2: 16}, {1: 11}]",131.0,NaN,NaN,"[{'Mon': '11:00 am - 1:00 am'}, {'Tue': '11:00..."
2,"2111 W Armitage Ave Chicago, IL 60647 b/t Leav...","Cuban,Latin American",Claimed,2005-08-19,NaN,101_cafe-laguardia,"[{'Takes Reservations': 'Yes'}, {'Delivery': '...",2014-06-08,41.917521,-87.680450,...,Bucktown,1,(773) 862-5996,$11-30,3.5,"[{5: 95}, {4: 187}, {3: 81}, {2: 43}, {1: 27}]",433.0,https://www.yelp.com/biz/cafe-laguardia-chicago,http://www.cafelaguardia.com,[]
3,"871 N Rush St Chicago, IL 60686 Near North Side",Italian,NaN,2005-11-02,NaN,103_cafe-luciano,[],2015-10-15,41.898765,-87.626551,...,Near North Side,1,(312) 266-1414,NaN,3.5,"[{5: 1}, {4: 3}, {3: 0}, {2: 0}, {1: 1}]",5.0,https://www.yelp.com/biz/cafe-luciano-chicago,NaN,[]
4,"1846 N Milwaukee Ave Chicago, IL 60647 b/t Blo...",French,Claimed,2005-11-08,NaN,104_cafe-matou,"[{'Takes Reservations': 'Yes'}, {'Delivery': '...",2011-04-12,41.914953,-87.684838,...,Bucktown,1,(773) 384-8911,$31-60,4.0,"[{5: 30}, {4: 44}, {3: 15}, {2: 8}, {1: 2}]",99.0,https://www.yelp.com/biz/cafe-matou-chicago,http://www.cafematou.com,[]
5,"4749 N Spaulding Ave Chicago, IL 60625 b/t Lel...",Coffee & Tea,Claimed,2015-09-24,NaN,105_cafe-chien,"[{'Take-out': 'Yes'}, {'Accepts Credit Cards':...",2017-08-04,41.968092,-87.710801,...,Albany Park,1,(312) 493-2658,Inexpensive,4.5,"[{5: 30}, {4: 12}, {3: 3}, {2: 1}, {1: 0}]",46.0,NaN,http://www.cafechien.com,[]
6,"4729 N Lincoln Ave Chicago, IL 60625 b/t Lelan...","Bakeries,Breakfast & Brunch,Cafes",Claimed,2005-07-13,NaN,106_cafe-selmarie,"[{'Takes Reservations': 'No'}, {'Delivery': 'N...",2017-08-09,41.967456,-87.687520,...,"Ravenswood, Lincoln Square",0,(773) 989-5595,$11-30,4.0,"[{5: 237}, {4: 272}, {3: 72}, {2: 35}, {1: 20}]",636.0,https://www.yelp.com/biz/cafe-selmarie-chicago,http://www.cafeselmarie.com,"[{'Mon': 'Closed'}, {'Tue': '8:00 am - 9:00 pm..."
7,"225 W Wacker Dr Chicago, IL 60606 b/t Post Pl ...","Italian,Caterers,Cafes",Claimed,2005-09-04,NaN,107_caffe-baci,"[{'Takes Reservations': 'No'}, {'Delivery': 'Y...",2017-07-27,41.886508,-87.634999,...,The Loop,0,(312) 251-0135,$11-30,3.5,"[{5: 11}, {4: 14}, {3: 10}, {2: 7}, {1: 3}]",45.0,https://www.yelp.com/biz/caffe-baci-chicago,http://www.caffebaci.com,"[{'Mon': '6:30 am - 9:00 pm'}, {'Tue': '6:30 a..."
8,"633 N St Clair St Chicago, IL 60610 Near North...","Italian,American (Traditional)",NaN,2005-07-22,NaN,108_caliterra,"[{'Takes Reservations': 'Yes'}, {'Take-out': '...",2013-03-10,41.893792,-87.622414,...,"Near North Side, River East",1,(312) 274-4444,$31-60,3.5,"[{5: 5}, {4: 8}, {3: 8}, {2: 5}, {1: 2}]",28.0,NaN,NaN,[]
9,"167 E Ohio St Chicago, IL 60611 b/t Michigan A...",Restaurants,NaN,2005-07-15,NaN,109_cambridge-house-ltd,[],2010-08-19,41.892534,-87.623114,...,"Near North Side, River East",1,(312) 828-0600,NaN,3.0,"[{5: 0}, {4: 3}, {3: 1}, {2: 0}, {1: 2}]",6.0,NaN,NaN,[]


In [361]:
Res.shape

(561, 21)

In [362]:
Res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 21 columns):
address               561 non-null object
category              544 non-null object
claimed_status        345 non-null object
first_review          561 non-null object
health_rating         0 non-null float64
id                    561 non-null object
info                  561 non-null object
last_review           561 non-null object
latitude              561 non-null float64
longitude             561 non-null float64
name                  561 non-null object
neighborhood          557 non-null object
permanently_closed    561 non-null int64
phone                 547 non-null object
price_range           467 non-null object
ratings               561 non-null float64
ratings_histogram     561 non-null object
reviews               516 non-null float64
url                   456 non-null object
website               377 non-null object
working_hours         561 non-null object
dtypes: flo

In [363]:
Closed = Res[Res['permanently_closed'] == 1]
Closed.shape

(479, 21)

In [364]:
ClosedandReviewed = Closed[(Closed['last_review'] != 'MISSING') & (Closed['first_review'] != 'MISSING') ]
ClosedandReviewed.shape

(453, 21)

In [365]:
ClosedRestaurants = ClosedandReviewed.sort_values(by='last_review', ascending=False)[:304]  #last_review back to 2012

In [366]:
ClosedRestaurants

,address,category,claimed_status,first_review,health_rating,id,info,last_review,latitude,longitude,...,neighborhood,permanently_closed,phone,price_range,ratings,ratings_histogram,reviews,url,website,working_hours
333,"300 W 24th St Chicago, IL 60616 b/t Calumet Av...",Italian,NaN,2007-07-30,NaN,50_bertuccis-corner,"[{'Takes Reservations': 'Yes'}, {'Delivery': '...",2017-08-31,41.849393,-87.634840,...,Chinatown,1,(312) 225-2848,$11-30,4.0,"[{5: 17}, {4: 16}, {3: 8}, {2: 1}, {1: 4}]",46.0,NaN,NaN,[]
531,"2834 N Southport Ave Chicago, IL 60657 b/t Wol...","French,American (New)",Claimed,2015-02-07,NaN,73_mirador,"[{'Takes Reservations': 'Yes'}, {'Delivery': '...",2017-08-24,41.933505,-87.663802,...,Lakeview,1,(773) 904-7431,$11-30,4.5,"[{5: 30}, {4: 8}, {3: 3}, {2: 4}, {1: 0}]",45.0,https://www.yelp.com/biz/mirador-chicago,http://miradorrestaurant.com/,[]
491,"923 W Weed St Chicago, IL 60642 Near North Side","Barbeque,Music Venues",Claimed,2015-04-18,NaN,35_dinosaur-bar-b-que,"[{'Takes Reservations': 'Yes'}, {'Delivery': '...",2017-08-07,41.909549,-87.652034,...,Near North Side,1,(312) 462-1053,$11-30,3.5,"[{5: 69}, {4: 68}, {3: 42}, {2: 41}, {1: 15}]",235.0,https://www.yelp.com/biz/dinosaur-bar-b-que-ch...,http://www.dinosaurbarbque.com/bbq-chicago,[]
5,"4749 N Spaulding Ave Chicago, IL 60625 b/t Lel...",Coffee & Tea,Claimed,2015-09-24,NaN,105_cafe-chien,"[{'Take-out': 'Yes'}, {'Accepts Credit Cards':...",2017-08-04,41.968092,-87.710801,...,Albany Park,1,(312) 493-2658,Inexpensive,4.5,"[{5: 30}, {4: 12}, {3: 3}, {2: 1}, {1: 0}]",46.0,NaN,http://www.cafechien.com,[]
216,"67 E Cermak Rd Chicago, IL 60616 b/t Federal S...",Smokehouse,Claimed,2016-10-19,NaN,371_rylons-smokehouse,"[{'Take-out': 'No'}, {'Accepts Credit Cards': ...",2017-07-29,41.852637,-87.624399,...,Near Southside,1,(312) 794-5901,Moderate,3.0,"[{5: 10}, {4: 12}, {3: 10}, {2: 9}, {1: 18}]",59.0,NaN,http://www.RylonsSmokehouse.com,[]
309,"1301 N State Pkwy Chicago, IL 60610 b/t Banks ...",American (New),Claimed,2011-09-16,NaN,486_pump-room,"[{'Takes Reservations': 'Yes'}, {'Delivery': '...",2017-07-29,41.906050,-87.628317,...,"Gold Coast, Near North Side",1,(312) 787-3700,$31-60,3.5,"[{5: 214}, {4: 248}, {3: 99}, {2: 78}, {1: 46}]",685.0,NaN,http://www.pumproom.com/,[]
153,"1901 N Halsted St Chicago, IL 60614 b/t Armita...","Vegetarian,Vegan,Live/Raw Food",Claimed,2006-06-23,NaN,288_karyns-cooked,"[{'Takes Reservations': 'Yes'}, {'Delivery': '...",2017-07-23,41.916034,-87.648141,...,Lincoln Park,1,(312) 255-1590,$11-30,3.0,"[{5: 76}, {4: 66}, {3: 44}, {2: 45}, {1: 52}]",283.0,https://www.yelp.com/biz/karyns-cooked-chicago,https://karynraw.com/restaurants/raw-bistro/,[]
164,"4662 N Broadway Chicago, IL 60640 b/t Leland A...",American (New),Claimed,2014-01-11,NaN,2_42-grams,"[{'Takes Reservations': 'Yes'}, {'Delivery': '...",2017-07-17,41.967116,-87.658568,...,Uptown,1,NaN,Above $61,4.5,"[{5: 64}, {4: 6}, {3: 5}, {2: 1}, {1: 4}]",80.0,https://www.yelp.com/biz/42-grams-chicago,http://42GramsChicago.com,[]
517,"5624 N Broadway Ave Chicago, IL 60660 b/t Bryn...","Pizza,Italian",Claimed,2007-07-17,NaN,5_apart-pizza-company,"[{'Takes Reservations': 'No'}, {'Delivery': 'Y...",2017-07-13,41.984456,-87.660380,...,Edgewater,1,NaN,$11-30,3.5,"[{5: 110}, {4: 83}, {3: 34}, {2: 28}, {1: 42}]",297.0,https://www.yelp.com/biz/apart-pizza-company-c...,http://www.apartpizzacompany.com,[]
66,"15 W Hubbard St Chicago, IL 60654 Near North Side","Mexican,Bars",Claimed,2010-04-27,NaN,176_dos-diablos,"[{'Takes Reservations': 'Yes'}, {'Delivery': '...",2017-06-25,41.890008,-87.628553,...,Near North Side,1,(312) 245-3100,$11-30,3.0,"[{5: 23}, {4: 56}, {3: 59}, {2: 47}, {1: 33}]",218.0,https://www.yelp.com/biz/dos-diablos-chicago,http://www.dosdiabloschicago.com,[]


In [367]:
ClosedRestaurants.to_csv('ClosedRestaurants.csv', index=False)

# Find out the neighborhoods in which closed restaurants located

In [368]:
ClosedRestaurants['neighborhood']

333                       Chinatown
531                        Lakeview
491                 Near North Side
5                       Albany Park
216                  Near Southside
309     Gold Coast, Near North Side
153                    Lincoln Park
164                          Uptown
517                       Edgewater
66                  Near North Side
279          West Town, Wicker Park
514                 Near North Side
202                 Near North Side
36     Near North Side, River North
537    Near North Side, River North
135            Lincoln Park, DePaul
512            Lincoln Park, DePaul
174                  Garfield Ridge
398          West Town, Wicker Park
281                 Near North Side
63             DePaul, Lincoln Park
161                    Lincoln Park
433                        Lakeview
532          Wicker Park, West Town
231                 Near North Side
363                 Near North Side
490                        Lakeview
40     River North, Near Nor

# This dictionary is used to grab open restaurants in project_capstone/part_02/OpenRestaurants/GetOpenRestaurantsYelpApi.ipynb

In [369]:
neighborhoods = {}
for nh in ClosedRestaurants['neighborhood']:
    if type(nh) != float: #to exclude nan as a float
        nh_list = nh.split(', ')
        for item in nh_list:
            if item not in neighborhoods.keys():
                neighborhoods[item] = 1
            else:
                neighborhoods[item] += 1
neighborhoods

{'Albany Park': 2,
 'Andersonville': 11,
 'Avondale': 3,
 'Bucktown': 12,
 'Chinatown': 4,
 'Cragin': 1,
 'DePaul': 19,
 'Edgewater': 10,
 'Forest Glen': 1,
 'Fulton Market': 3,
 'Garfield Ridge': 1,
 'Gold Coast': 2,
 'Greektown': 2,
 'Humboldt Park': 2,
 'Irving Park': 5,
 'Lakeview': 40,
 'Lincoln Park': 31,
 'Lincoln Square': 4,
 'Logan Square': 12,
 'Near North Side': 80,
 'Near Southside': 5,
 'Near West Side': 22,
 'Noble Square': 7,
 'North Center': 8,
 'Old Town': 8,
 'Pilsen': 1,
 'Portage Park': 2,
 "Printer's Row": 2,
 'Ravenswood': 6,
 'River East': 7,
 'River North': 27,
 'River West': 2,
 'Rogers Park': 3,
 'Roscoe Village': 1,
 'South Loop': 3,
 'Streeterville': 2,
 'The Loop': 7,
 'Ukrainian Village': 3,
 'University Village': 4,
 'Uptown': 8,
 'West Loop': 21,
 'West Town': 36,
 'Wicker Park': 23,
 'Wrigleyville': 1}

In [370]:
len(neighborhoods)

44

In [371]:
list(neighborhoods.keys())

['Chinatown',
 'Lakeview',
 'Near North Side',
 'Albany Park',
 'Near Southside',
 'Gold Coast',
 'Lincoln Park',
 'Uptown',
 'Edgewater',
 'West Town',
 'Wicker Park',
 'River North',
 'DePaul',
 'Garfield Ridge',
 'Streeterville',
 'Logan Square',
 'Bucktown',
 'Ukrainian Village',
 'Old Town',
 'Ravenswood',
 'Lincoln Square',
 'Wrigleyville',
 'Irving Park',
 'River East',
 'Andersonville',
 'Near West Side',
 'West Loop',
 'Greektown',
 'Avondale',
 'River West',
 'Noble Square',
 'The Loop',
 'North Center',
 'Rogers Park',
 'Portage Park',
 'Fulton Market',
 'Forest Glen',
 'Pilsen',
 'South Loop',
 'Cragin',
 'Humboldt Park',
 'University Village',
 "Printer's Row",
 'Roscoe Village']

neighborhoods: 
['Chinatown',
 'Lakeview',
 'Near North Side',
 'Albany Park',
 'Near Southside',
 'Gold Coast',
 'Lincoln Park',
 'Uptown',
 'Edgewater',
 'West Town',
 'Wicker Park',
 'River North',
 'DePaul',
 'Garfield Ridge',
 'Streeterville',
 'Logan Square',
 'Bucktown',
 'Ukrainian Village',
 'Old Town',
 'Ravenswood',
 'Lincoln Square',
 'Wrigleyville',
 'Irving Park',
 'River East',
 'Andersonville',
 'Near West Side',
 'West Loop',
 'Greektown',
 'Avondale',
 'River West',
 'Noble Square',
 'The Loop',
 'North Center',
 'Rogers Park',
 'Portage Park',
 'Fulton Market',
 'Forest Glen',
 'Pilsen',
 'South Loop',
 'Cragin',
 'Humboldt Park',
 'University Village',
 "Printer's Row",
 'Roscoe Village']

In [372]:
for nh in list(neighborhoods.keys()):
    print(nh)
    #ClosedRestaurants[nh] = ClosedRestaurants['neighborhood'].apply(lambda x: 1 if nh in str(x) else 0)

Chinatown
Lakeview
Near North Side
Albany Park
Near Southside
Gold Coast
Lincoln Park
Uptown
Edgewater
West Town
Wicker Park
River North
DePaul
Garfield Ridge
Streeterville
Logan Square
Bucktown
Ukrainian Village
Old Town
Ravenswood
Lincoln Square
Wrigleyville
Irving Park
River East
Andersonville
Near West Side
West Loop
Greektown
Avondale
River West
Noble Square
The Loop
North Center
Rogers Park
Portage Park
Fulton Market
Forest Glen
Pilsen
South Loop
Cragin
Humboldt Park
University Village
Printer's Row
Roscoe Village
